# 03. Notebook: Sentiment RNN

영화 리뷰들을 분석하여 해당 리뷰가 긍정적 리뷰인지 / 부정적 리뷰인지 예측하는 모델만들기

---

In [1]:
import os
os.getcwd()

'C:\\Users\\USER\\Desktop\\Sentiment_Analysis'

## 01. Data pre-processing

### 1-0.Load in and visualize the data

In [2]:
import numpy as np

# Read data from text files
with open('data/reviews.txt', 'r') as f:
    reviews = f.read()
    
with open('data/labels.txt', 'r') as f:
    labels = f.read()

In [3]:
print(f"The number of Review's characters: {len(reviews)}")
print()
print(f"Review data sample: \n {reviews[:2000]}")
print()
print(f"Label data sample: \n {labels[:100]}")

The number of Review's characters: 33678267

Review data sample: 
 bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terr

**Comment** :



대략 3천3백만개의 글자로 이루어진 리뷰 텍스트 데이터가 있고


각 리뷰에 대한 긍정/부정 정답, 즉 라벨데이터도 단순 텍스트덩어리로 되어있구나


대충 리뷰를 읽어보니 Airport77이라는 영화의 리뷰인거같고 각 리뷰간의 구분은 `\n`, 즉 줄바꿈으로 되는거같다

### 1-1. Data pre-processing

텍스트데이터 -> 단어

단어 -> 정수 (with Embedding layer)가 되어야 RNN에 사용할 수 있음

In [4]:
# 텍스트 정제
# 소문자로 변경, 구두점 삭제
from string import punctuation
print(punctuation)

reviews = reviews.lower()
all_text = ''.join([c for c in reviews if c not in punctuation])

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [5]:
# 리뷰간 구분하여 리스트만들기, 즉 [리뷰1, 리뷰2, 리뷰3...]
reviews_split = all_text.split('\n')

# 딕셔너리 만들기 위해!
# 리스트로 쪼개진 리뷰들을 띄어쓰기로 구분하여 다시 덩어리로 만들기
all_text = ' '.join(reviews_split)

In [6]:
# 단어 리스트로 만들기 (리뷰간 구분없는)
words = all_text.split()
words[:10]

['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the']

### 1-2. Encoding the words

Embedding Lookup Table을 사용하려면 입력이 '정수값'이어야 하므로 단어<-> 정수 Dictionary를 만들자 (빈도수순으로 정렬해서~)

In [7]:
from collections import Counter
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)

<span style="color:red">[!]</span> 추후에 패딩을 '0'으로 할것이므로 겹치지 않게 정수값 시작을 '1'부터로 하자

In [8]:
vocab_to_int = { word: ii for ii, word in enumerate(vocab, start=1)}

리뷰 단어 -> 정수로 변환

아까 위에서 만들어놨던 (정제된) 리뷰 리스트 `reviews_split` [리뷰1, 리뷰2, 리뷰3...]을 이제 정수로 변환해주자

* [리뷰1, 리뷰2, ...] 리스트를 for loop로 돌린다
* 하나씩 나오는 리뷰를 `split()`을 통해 단어가 나오게끔 for loop로 돌린다
* 튀어나오는 단어들을 딕셔너리로 변환하여 리스트에 쌓는다
* 결론적으로 [[리뷰1단어1, 리뷰1단어2,...], [리뷰2단어1, 리뷰2단어2], ....]와 같은 이중 리스트가 됨

In [9]:
reviews_ints = []

for review in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])

### 1-3. Encoding the labels

아래처럼 각 리뷰에 대한 긍정/부정 라벨을 쪼개고, 

Positive는 1, Negative는 0으로 바꿔주자

In [10]:
labels[:50]

'positive\nnegative\npositive\nnegative\npositive\nnegat'

In [11]:
labels_split = labels.split('\n')
encoded_labels = [1 if label == 'positive' else 0 for label in labels_split]
print(encoded_labels[:10])

[1, 0, 1, 0, 1, 0, 1, 0, 1, 0]


### 1-4. Removing Outliers

<span style="color:green">Additional pre-processing</span> -> `reviews_ints`는 아래와 같은 이차원(?) 리스트임

[[리뷰1단어1, 리뷰1단어2,...],[리뷰2단어1, 리뷰2단어2,...],....]

주의할 것이 각 리뷰마다의 길이, 즉 단어의 개수가 다르므로 이를 맞춰줄 필요가 있음

따라서 단어개수가 하나도 없는 리뷰 or 너~무 많은 리뷰는 어느정도 제거해줘야함

In [12]:
# 각 리뷰의 길이 추출
len_review = [ len(r) for r in reviews_ints]
print(f"minimum review length : {min(len_review)}")
print(f"Maximum review length : {max(len_review)}")

minimum review length : 0
Maximum review length : 2514


<span style="color:red">[!]</span> 길이가 0인, 즉 단어개수가 0인, 즉 아무것도 안써져있는 Review가 있나보네, 제일 긴 리뷰는 단어개수가 무려 2,514개

In [13]:
# 길이(단어개수)가 0이 아닌 리뷰의 인덱스만 추출하기
non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review) != 0]
reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]

In [14]:
# 길이(단어개수)가 0이 아닌 리뷰의 라벨만 추출하기
encoded_labels = [encoded_labels[ii] for ii in non_zero_idx]

---

## 02. Padding Sequences

* `reviews_ints` : 이중 리스트, 바깥은 리뷰단위, 속은 리뷰의 단어(정수)단위

[[리뷰1단어1, 리뷰1단어2,], [리뷰2단어1,리뷰2단어2,...]....]

그런데 각 리뷰마다 단어개수가 같을수없을거아니야? 우리는 RNN에 리뷰의 단어들을 Sequence느낌으로 넣어줘야하는데 이 Sequence Length가 동일해야겠지? 따라서

1. `seq_length`보다 짧은 리뷰는 왼쪽부터 0으로 채워넣기 (패딩)
2. `seq_length`보다 긴 리뷰는 `seq_length`까지만 남기고 나머지는 잘라내기


### 2-1. Padding or Truncate function

아래와 같은 입출력을 먹고뱉는 함수를 만들자 `pad_features`

입력 : `reviews_ints`, 즉 이중리스트, `seq_length`

출력 : `features`, Review개수 x Sequence Length 사이즈의 2D Array

In [18]:
def pad_features(reviews_ints, seq_length):
    '''
    Return features of reviews_ints, where each review is padded with 0's or
    truncated to the input seq_length
    '''
    
    review_li = []   # padded or truncated review 담아둘 리스트
    for r in reviews_ints:
        if len(r) < 200:
            padding_n = seq_length - len(r)
            padded_review = np.pad(r,
                                   (padding_n, 0),
                                   'constant',
                                   constant_values = 0
                                   )
            review_li.append(padded_review)
            
        elif len(r) >= 200:
            truncated_review = np.array(r[:seq_length])
            review_li.append(truncated_review)
            
    # review_li는 길이 200의 review array들이 있는 '리스트'
    # concatenate를 이용하여 일렬로 쭉 붙여주고
    # reshape을 통해 (리뷰개수, seq_length)로 사이즈 만들어주기
    features = np.concatenate(review_li).reshape(-1, seq_length)
    
    return features

### 2-2. Function Check

In [21]:
seq_length = 200

features = pad_features(reviews_ints, seq_length=seq_length)

assert len(features) == len(reviews_ints)
assert len(features[0]) == seq_length

print(features[:10, :10])
print()
print(features.shape)

[[    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [22382    42 46418    15   706 17139  3389    47    77    35]
 [ 4505   505    15     3  3342   162  8312  1652     6  4819]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [   54    10    14   116    60   798   552    71   364     5]]

(25000, 200)


**Comment** :

Row는 각각의 리뷰(나중에 batch로 쓰일)들이고
Column은 각 리뷰의 첫 10개의 단어(정수)

총 25000개의 리뷰, 그리고 (각 리뷰의) 200개의 단어들이 `features`에 
2D array로 할당되어있다

---

## 03. Training, Validation, Test

* `features` : `(review개수, seq_lengh)` 사이즈의 2D Array
* `encoded_labels` : `(, review개수)` 사이즈의 1D Array

위 두개의 Array를 아래와 같은 비율로 나눠주자

* Training : 80% `train_x` `train_y`
* Validation : 10% `val_x` `val_y`
* Test : 10% `test_x` `test_y`

In [31]:
# encoded_labels array로 변환시키기
encoded_labels = np.array(encoded_labels)

In [32]:
split_frac = 0.8


# Training / Val+Test 나누기
split_idx = int(len(features)*split_frac)  # 전체길이의 80%, 정수값 인덱스
train_x, valtest_x = features[:split_idx], features[split_idx:]
train_y, valtest_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

#------------------------------------------------------------------

# Val Test 나누기
split_idx_2 = int(len(valtest_x)*0.5)      # 전체길이의 20%의 절반, 즉 전체길이의 10%
val_x, test_x = valtest_x[:split_idx_2], valtest_x[split_idx_2:]
val_y, test_y = valtest_y[:split_idx_2], valtest_y[split_idx_2:]


In [33]:
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape),
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


---

## 04. DataLoaders and Batching

* `train,valid,test_x` : x 데이터셋 (리뷰개수, 각리뷰 단어) 2D array
* `train,valid,test_y` : y 데이터셋 (리뷰개수) 각 리뷰의 라벨 1D array

데이터셋 준비는 완료되었다.

`torch.utils.data.TensorDataset`을 통하여 x,y 텐서 데이터셋을 만들고
(모아놓은) 데이터셋을 통하여 DataLoader를 만들자

### 4-1. DataLoader 만들기

In [35]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Create TensorDatasets
train_data = TensorDataset(torch.from_numpy(train_x),
                           torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x),
                           torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x),
                          torch.from_numpy(test_y))

In [36]:
# Create DataLoader
batch_size = 50
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

### 4-2. 1개의 Batch Check

In [39]:
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print(f"Sample input size: {sample_x.size()}")
print(f"Sample input: \n {sample_x}")
print()
print(f"Sample label size: {sample_y.size()}")
print(f"Sample label: \n {sample_y}")

Sample input size: torch.Size([50, 200])
Sample input: 
 tensor([[    0,     0,     0,  ...,   881,     6,   898],
        [    0,     0,     0,  ...,   236,   115,    11],
        [   54,     3,    18,  ...,     5,  3207,    42],
        ...,
        [  594, 12641,    47,  ...,  2255,  1522,     2],
        [52491,   125,    48,  ...,    97,    28,    77],
        [    1,   421,     4,  ...,     1,   219,   672]], dtype=torch.int32)

Sample label size: torch.Size([50])
Sample label: 
 tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1,
        0, 1], dtype=torch.int32)


<span style="color:red">[!]</span> input과 label의 Row는 하나의 리뷰를 의미하고 input에서의 Col은 Sequence Length, 즉 모델에 들어갈 단어의 개수를 의미한다. 또한 지금 모든 텐서가 int32 타입임을 인지하자

---

## 05. Network Architecture

데이터셋 만들기 완료, DataLoader까지도 만들기 완료

이제 드디어 '모델'을 만들 차례이다. 모델의 Layer는 아래와 같이 만들예정

1. Embedding Layer : 단어 Token을 특정 크기의 Embedding vector로 변환해줌
2. LSTM Layer : `hidden_state`의 size, 즉 LSTM cell의 노드수와 층과 층으로 이루어짐
3. Fully-Connected Layer : LSTM Layer의 출력사이즈를 우리가 원하는 출력사이즈로 바꿔줌
4. Sigmoid Activation Layer : 모든 출력들을 0~1사이의 값으로 바꿔줌

### 5-1. The Embedding Layer

사용할 데이터셋(영화리뷰)에는 Unique Vocab이 무려 74,000개나 있고 이를 원핫인코딩, 즉 희소벡터로 만들어 사용하는 것은 매우 비효율적이므로 임베딩 레이어를 사용해야 한다.

### 5-2. The LSTM Layer

LSTM layer는 아래의 parameter를 받는다

* `input_size` : 임베딩 레이어를 지나 만들어진 임베딩 벡터가 들어올 예정이므로 embedding dimension

* `hidden_dim` : LSTM cell의 Hidden layer의 유닛개수, 보통 128, 256, 512로 함

* `number of layers` : Hidden Layer의 개수

* `dropout probability`

#### GPU Check

In [40]:
train_on_gpu = torch.cuda.is_available()

if (train_on_gpu):
    print("Training on GPU")
else:
    print("No GPU available, Training on CPU")

No GPU available, Training on CPU


### 5-3. Architecture

In [44]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment Analysis
    """
    
    def __init__(self,
                 vocab_size,    # 우리가 넣어줄 입력 차원 = Unique vocab 개수
                 output_size,
                 embedding_dim, # 임베딩 벡터의 길이
                 hidden_dim,
                 n_layers,
                 drop_prob=0.5):
    
        super(SentimentRNN, self).__init__()
        
        # 왜 얘네만 self 선언하지?
        # self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # Embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                            dropout=drop_prob, batch_first=True)
        
        # Dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # Linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        
        
    def forward(self, x, hidden):
        
        batch_size = x.size(0)    # input으로 들어온 배치덩어리의 Row수
        
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        # Stack up LSTM outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        sig_out = self.sig(out)
        
        
        # Reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        # 출력에서 마지막 Column의 값, 즉 최종 타임스텝의 output값만 가져오기
        sig_out = sig_out[:, -1]
        
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        """
        Initializes hidden state
        """
        
        # Create two new tensors with sizes(n_layers, batch_size, hidden_dim)
        # Initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers,
                                 batch_size,
                                 self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers,
                                 batch_size,
                                 self.hidden_dim).zero_().cuda()
                     )
            
        else:
            hidden = (weight.new(self.n_layers,
                                 batch_size,
                                 self.hidden_dim).zero_(),
                      weight.new(self.n_layers,
                                 batch_size,
                                 self.hidden_dim).zero_()
                     )
            
        return hidden

---

## 06. Instantiate the network

* `vocab_size` : 우리가 모델에 넣는 데이터(최소단위)의 값의 개수는? Unique vocab개수! + 1 (for padding)
* `output_size` : The number of class score, 영화리뷰에 대한 감정도
* `embedding_dim` : Number of columns in the embedding lookup table; size of our embedding vector
* `hidden_dim` : Number of units in the hidden layers of our LSTM class
* `n_layers` : 

### 6-1. Instantiate the model

In [49]:
vocab_size = len(vocab_to_int) +1
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size,
                   output_size,
                   embedding_dim,
                   hidden_dim,
                   n_layers)

In [50]:
net

SentimentRNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)

**Comment** :

우리의 영화리뷰 데이터의 Unique한 Vocabulary의 개수는 74,072개임.

여기에 Padding을 의미하는 '0'도 포함시켜서 총 74,073개

이걸 Embedding layer에 먼저 넣어서 400 차원으로 축소시킴

LSTM layer를 통해서 256개의 unit값

Dropout layer로 30%는 꺼버리고

마지막 FC layer를 통해서 1개의 값으로 만듦

그 값을 Sigmoid로 압축

---

## 07. Training

### 7-1. Loss and Optimization

In [51]:
lr = 0.001
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

### 7-2. Training

In [52]:
# Training time 측정용
import time
from time import time

In [56]:
epochs = 4
counter = 0
print_every = 100
clip = 5

# Move model to GPU, if available
if (train_on_gpu):
    net.cuda()
    
net.train()

for e in range(epochs):
    
    start = time()
    # Initialize hidden state
    h = net.init_hidden(batch_size)
    
    for inputs, labels in train_loader:   # 50개의 batch 총 400번 = 20000
        counter += 1
        
        # Move dataset to GPU, if available
        if (train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()
            
        # Hidden state를 새로운 변수에 복사 및 떼놓기
        # .data를 통해서 requires_grad=False가 되면서 Hidden state가
        # 굳이 '다시' 학습되지 않는다
        h = tuple([each.data for each in h])
        
        # Zero accumulated gradients
        net.zero_grad()
        
        # Get the output from the model
        output, h = net(inputs,h)
        
        # Calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()
        
        # Loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:
                val_h = tuple([each.data for each in val_h])
                
                if (train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()
                    
                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())
                
                val_losses.append(val_loss.item())
               
            end = time()
            net.train()
            print(f"100step time: {end-start} [sec]")
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

100step time: 234.29950976371765 [sec]
Epoch: 1/4... Step: 100... Loss: 0.590224... Val Loss: 0.647421
100step time: 384.726930141449 [sec]
Epoch: 1/4... Step: 200... Loss: 0.575770... Val Loss: 0.580677
100step time: 533.2854495048523 [sec]
Epoch: 1/4... Step: 300... Loss: 0.646433... Val Loss: 0.593216
100step time: 679.8651714324951 [sec]
Epoch: 1/4... Step: 400... Loss: 0.566177... Val Loss: 0.587839
100step time: 148.69187378883362 [sec]
Epoch: 2/4... Step: 500... Loss: 0.697694... Val Loss: 0.692770
100step time: 294.4412753582001 [sec]
Epoch: 2/4... Step: 600... Loss: 0.705943... Val Loss: 0.690536
100step time: 440.5910835266113 [sec]
Epoch: 2/4... Step: 700... Loss: 0.526810... Val Loss: 0.617581
100step time: 587.9177124500275 [sec]
Epoch: 2/4... Step: 800... Loss: 0.404018... Val Loss: 0.513853
100step time: 146.95251655578613 [sec]
Epoch: 3/4... Step: 900... Loss: 0.324925... Val Loss: 0.486536
100step time: 298.1655373573303 [sec]
Epoch: 3/4... Step: 1000... Loss: 0.482657

---

## 08. Testing


### 8-1. Testing with Test data

In [86]:
test_losses = []
num_correct = 0

# Init hidden state
h = net.init_hidden(batch_size)

net.eval()

for inputs, labels in test_loader:     # 50개의 batch 총 50번 = 2500
    h = tuple([each.data for each in h])
    
    if (train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
        
    output, h = net(inputs,h)
    
    test_loss = criterion(output.squeeze(), labels.float())  # 매 Batch마다의 loss
    test_losses.append(test_loss.item())                     # 누적시키기
    
    # output을 1또는 0으로 변환(가까운 값으로, 반올림?)
    pred = torch.round(output.squeeze())
    
    # 정답(label)값과 반올림된것 비교
    # tensor.eq(Tensor A, Tensor B) = 각 위치의 값이 같으면 True, 다르면 False
    # 로 이루어진 Tensor 반환
    correct_tensor = pred.eq(labels.float().view_as(pred))
    
    # Tensor를 NumPy Array 변환
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    
    # 맞춘값 누적
    num_correct += np.sum(correct)
    

# Average Test loss
print(f"Test loss: {np.mean(test_losses)}")

# Accuracy over all test data
test_acc = num_correct / len(test_loader.dataset)   # 2500, 즉 배치개수가 아닌 전체 데이터 개수
    
print(f"Test Accuracy: {test_acc}")

Test loss: 0.48386398822069165
Test Accuracy: 0.7936


[+] 모델저장

In [87]:
# Model save
torch.save(net.state_dict(), "SentimentRNN_epoch4.pt")

# Model load
net2 = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
net2.load_state_dict(torch.load("SentimentRNN_epoch4.pt"))
net2.eval()

SentimentRNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)

### 8-2. Inference

아래의 두가지 리뷰를 모델에 넣어서 어떻게 예측하는지 살펴보자

In [93]:
test_review_neg = "The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow."
                   
test_review_pos = "This movie had the best acting and the dialogue was so good. I loved it."

#### (1) 정제작업

In [94]:
from string import punctuation
test_review = test_review_neg.lower()
test_text = ''.join([c for c in test_review if c not in punctuation])
print(test_text)

the worst movie i have seen acting was terrible and i want my money back this movie had bad acting and the dialogue was slow


#### (2) 텍스트 -> 단어리스트

In [95]:
test_words = test_text.split()
print(test_words)

['the', 'worst', 'movie', 'i', 'have', 'seen', 'acting', 'was', 'terrible', 'and', 'i', 'want', 'my', 'money', 'back', 'this', 'movie', 'had', 'bad', 'acting', 'and', 'the', 'dialogue', 'was', 'slow']


#### (3) 단어 -> 정수

In [96]:
test_ints = [vocab_to_int[word] for word in test_words]

Batch, Review's' 차원으로 만들기 -> 우리가 `pad_features()` 함수를 이용하여 Padding을 진행할때 입력으로 받은 데이터는 Row는 리뷰개수, Column은 1개의 리뷰내의 단어개수의 차원이었다. 따라서 지금은 1개의 리뷰이지만 2차원으로 만들어줘야 패딩을 진행할 수 있다

In [97]:
test_ints = [test_ints]

#### (1,2,3) Function

In [102]:
from string import punctuation

def tokenize_review(test_review):
    test_review = test_review.lower()
    test_text = ''.join([c for c in test_review if c not in punctuation])
    test_words = test_text.split()
    test_ints = [vocab_to_int[word] for word in test_words]
    test_ints = [test_ints]
    
    return test_ints

#### Prediction Function

In [109]:
def predict(net, test_review, sequence_length=200):
    '''
    net : A Trained Network model
    test_review : A review made of normal text and punctuation
    sequence_length : The padded length of a review
    '''
    
    net.eval()
    
    test_ints = tokenize_review(test_review)
    
    seq_length = sequence_length 
    
    features = pad_features(test_ints, seq_length)
    feature_tensor = torch.from_numpy(features)
    
    
    batch_size = feature_tensor.size(0)
    
    h = net.init_hidden(batch_size)
    
    if (train_on_gpu):
        feature_tensor = feature_tensor.cuda()
        
    output, h = net(feature_tensor, h)
    pred = torch.round(output.squeeze())
    
    print(f"Prediction value, (pre-rounding): {output.item()})")
    
    if (pred.item() == 1):
        print("Positive Review!")
    else:
        print("Negative Review!")

In [110]:
predict(net2, test_review_neg)

Prediction value, (pre-rounding): 0.01452025305479765)
Negative Review!


In [111]:
predict(net2, test_review_pos)

Prediction value, (pre-rounding): 0.9390115737915039)
Positive Review!
